In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install datasets

In [3]:
!pip install gensim nltk


In [4]:
import datasets
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import gensim.downloader as api
import nltk
from nltk.stem import WordNetLemmatizer
import torch
import torch.nn as nn
from tqdm.notebook import tqdm, trange
from nltk.tokenize import WordPunctTokenizer
from sklearn.utils.class_weight import compute_class_weight


In [5]:
nltk.download('wordnet')
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/co

In [6]:
df_all = pd.read_csv('/kaggle/input/cleaned-toxic-comments/train_preprocessed.csv')[['comment_text', 'toxic']]

In [7]:
df_train = df_all.sample(frac = 0.85)
df_test = df_all.drop(df_train.index)

In [8]:
df_train['toxic'].value_counts(), df_test['toxic'].value_counts()

(toxic
 0.0    122606
 1.0     13029
 Name: count, dtype: int64,
 toxic
 0.0    21671
 1.0     2265
 Name: count, dtype: int64)

In [9]:
class_weights = torch.from_numpy(compute_class_weight('balanced', classes = np.unique(df_train['toxic'].values), y = df_train['toxic'].values))
class_weights

tensor([0.5531, 5.2051], dtype=torch.float64)

In [10]:
type(class_weights)

torch.Tensor

In [11]:
word2vec = api.load('glove-twitter-50')

[========================================----------] 81.2% 162.0/199.5MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[==================================================] 100.0% 199.5/199.5MB downloaded


In [13]:
word2vec.vectors.shape

(1193514, 50)

In [14]:
word2index = {word: index for index, word in enumerate(word2vec.index_to_key)}

In [15]:
class MyDataset(Dataset):
    def __init__(self, df):
        self.df = df.dropna(subset = 'comment_text')
        self.x = self.df['comment_text'].values
        self.y = self.df['toxic'].fillna(-1).values
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        return self.x[idx], torch.tensor(self.y[idx], dtype = torch.long)
        
train = MyDataset(df_train)
test = MyDataset(df_test)
    

In [16]:
# preprocess
def preprocess_dataset(dataset, tokenyzer, lemmatizer, mapp):
    for i in range(len(dataset.x)): 
        dataset.x[i] = tokenyzer.tokenize(dataset.x[i])
        dataset.x[i] = [lemmatizer.lemmatize(word) for word in dataset.x[i]]
        dataset.x[i] = torch.tensor([mapp[word] if word in mapp else mapp['unk'] for word in dataset.x[i]], 
                                   dtype = torch.long)
preprocess_dataset(train, WordPunctTokenizer(), WordNetLemmatizer(), word2index)
preprocess_dataset(test, WordPunctTokenizer(),  WordNetLemmatizer(), word2index)

In [19]:
train[1]

(tensor([   305,   1290,     85,    175,    137,    568,  15688,   4649,    229,
             96,     32,  62980,    137,    716,     39,     13,   6261,     26,
             13,    469,     32,     13,   1258,   2002,     58,  21479,     16,
             13,    325,     96,     13,   6762,  12865,     16,     56,     73,
             55,     13,   1411,     11,     58,    469,   6762,     32,   4530,
             33,     59,    775,  62980,    137,    716,     35,   7888,    153,
           1897,     33,    551,    716,     33,     35,  27155,     58,  21479,
             16,     13,   1258,   2002,     10,    186,  62980,     32,   5549,
             45,     13,  87271,     39,     13,  40226, 118290,     32,  20223,
         224031,     26,  44995,     26,     45,     74,     80,  10236,     45,
         118290,     80,    276,    894,  10236,     13, 206230,   1258,   2002,
             26,     13,  24479, 118290,     32,    175,     96,     13,  87271,
             39,     13,  40

In [18]:
test[0]

(tensor([   453,     15,    516,   5481, 134401,     26,   6655,     66,     21,
            265,   3053,     13,   6006,   7016,    744,   1638, 401454,   2819,
          57979,    187,    211,     16,  34349,     61,   7196,     33,    128,
          28544,    183,     33,    273,     63,     15,     70,  44367,     61,
           1426,     11,    172,   5072,    110,     13,   6006,   7016,     32,
          14027,     53,     32,    896,     13,    731,    449,   4393,    121,
           2466,  13313,    107,    615,     11,     39,    969,    249,  20608,
             32,  48035,    765,    107,     32,     13,    214,   1704,   1311,
           2683,    127,    195,     78,   6955,     45,    137,  10834,    145,
          48035,    285,    249,  71153,    172,  62980,  32456,     10,  37907,
             16, 132579,     53,   6006,     35,    420,     39,  75508,     13,
           1823,     39,    172,   5072,     45,     32,   5680,     16,    273,
             66,     13,   4

In [20]:
# writing function to handle data in batches
def collate_fn(batch):
    max_len = max([len(data_tup[0]) for data_tup in batch])
    input_embeddings = torch.empty((len(batch), max_len), dtype = torch.long)
    labels = torch.empty(len(batch), dtype = torch.long)
    for idx, data_tup in enumerate(batch):
        to_pad = max_len - len(data_tup[0])
        input_embeddings[idx] = torch.cat((data_tup[0], torch.zeros(to_pad)))
        labels[idx] = data_tup[1]
    return input_embeddings, labels

In [21]:
# creating DataLoaders to pass data in RNN,GRU,LSTM
train_loader = DataLoader(train, batch_size = 32, shuffle = True, collate_fn = collate_fn)
test_loader = DataLoader(test, batch_size = 32, shuffle = True, collate_fn = collate_fn)


loaders = {'train' : train_loader, 'test' : test_loader}

In [22]:
# writing out training function
def training(model, criterion, optimizer, num_epochs, loaders, max_grad_norm = 1):
    for e in trange(num_epochs, leave = False):
        model.train()
        num_iter = 0
        pbar = tqdm(loaders['train'], leave = False)
        for batch in pbar:
            optimizer.zero_grad()
            input_embeddings = batch[0].to(device)
            labels = batch[1].long().to(device)
            prediction = model(input_embeddings)
            loss = criterion(prediction, labels)
            loss.backward()
            if max_grad_norm is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            num_iter += 1
        valid_loss, valid_acc, num_iter = 0, 0, 0
        model.eval()
        with torch.no_grad():
            correct, num_objs = 0, 0
            for batch in loaders['test']:
                input_embeddings = batch[0].to(device)
                labels = batch[1].to(device)
                prediction = model(input_embeddings)
                valid_loss += criterion(prediction, labels)
                correct += (labels == prediction.argmax(-1)).float().sum()
                num_iter += 1
                num_objs += len(labels)
        print(f"Valid loss: {valid_loss / num_iter}, accuracy: {correct / num_objs}")
        

In [71]:
# RNN
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes = 2):
        super().__init__()
        self.hidden_size = hidden_size
        self.embeddings = nn.Embedding.from_pretrained(torch.from_numpy(word2vec.vectors))
        self.rnn = nn.RNN(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, num_classes)
    def forward(self, x):
        x = self.embeddings(x)
        x = x.permute(1, 0, 2)
        output, h_n = self.rnn(x)
        output = self.fc(h_n.squeeze(0)).float()
        return output

In [72]:
# initialize model
device = "cuda" if torch.cuda.is_available() else "cpu"
rnn = RNN(word2vec.vector_size, 100).to(device)
class_weights = class_weights.float().to(device)
criterion = nn.CrossEntropyLoss(weight = class_weights)
optimizer = torch.optim.Adam(rnn.parameters(), lr = 1e-2)
num_epochs = 10
max_grad_norm = 1.0

In [73]:
training(rnn, criterion, optimizer, num_epochs, loaders, max_grad_norm)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/4239 [00:00<?, ?it/s]

Valid loss: 0.6932706236839294, accuracy: 0.12052974849939346


  0%|          | 0/4239 [00:00<?, ?it/s]

Valid loss: 0.6930031180381775, accuracy: 0.8809325098991394


...

Valid loss: 0.6869073510169983, accuracy: 0.905372679233551


  0%|          | 0/4239 [00:00<?, ?it/s]

Valid loss: 0.9922727346420288, accuracy: 0.8486798405647278


  0%|          | 0/4239 [00:00<?, ?it/s]

Valid loss: 1.1134183406829834, accuracy: 0.8485127091407776


In [32]:
# lstm 
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes = 2):
        super().__init__()
        self.hidden_size = hidden_size
        self.embeddings = nn.Embedding.from_pretrained(torch.from_numpy(word2vec.vectors))
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, num_classes)
    def forward(self, x):
        x = self.embeddings(x).permute(1, 0, 2)
        output, (hn, cn) = self.lstm(x)
        output = self.fc(hn.squeeze(0)).float()
        return output

In [33]:
device = "cuda" if torch.cuda.is_available() else "cpu"
lstm = LSTM(word2vec.vector_size, 100).to(device)
class_weights = class_weights.float().to(device)
criterion = nn.CrossEntropyLoss(weight = class_weights)
optimizer = torch.optim.Adam(lstm.parameters(), lr = 1e-2)
num_epochs = 10
max_grad_norm = 1.0

In [34]:
training(lstm, criterion, optimizer, num_epochs, loaders, max_grad_norm)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/4239 [00:00<?, ?it/s]

Valid loss: 0.2815140187740326, accuracy: 0.9265123605728149


  0%|          | 0/4239 [00:00<?, ?it/s]

Valid loss: 0.27866724133491516, accuracy: 0.9217497110366821


...

Valid loss: 0.2331007719039917, accuracy: 0.9305648803710938


  0%|          | 0/4239 [00:00<?, ?it/s]

Valid loss: 0.2460559904575348, accuracy: 0.9105531573295593


  0%|          | 0/4239 [00:00<?, ?it/s]

Valid loss: 0.23309336602687836, accuracy: 0.9158172011375427


In [62]:
def predict_single(message, model, lemmatizer, tokenizer):
    model.eval()
    message = [lemmatizer.lemmatize(word) for word in tokenizer.tokenize(message)]
    message = torch.tensor([word2index[word] if word in word2index else word2index["unk"] for word in message])
    batched_message = message.unsqueeze(0).to(device)
    pred = model(batched_message)
    return 'toxic' if (torch.sigmoid(pred[0][0]) < torch.sigmoid(pred[0][1])).item() else 'normal'

In [65]:
pred = predict_single('It was great goal, probably best goal which i have ever seen', lstm, WordNetLemmatizer(), WordPunctTokenizer())

In [85]:
comms = ['yo whats up my friend',
         'how are you doing old fuck?', 
         "i guess you going to your trash work again didn't you?",
        "clean this trash can"]
for comm in comms:
    print(predict_single(comm, lstm, WordNetLemmatizer(), WordPunctTokenizer()))

normal
toxic
toxic
toxic


RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.cuda.FloatTensor instead (while checking arguments for embedding)